In [1]:
#make sure to build your environment:

#%pip install matplotlib numpy pandas plotly voila ipywidgets

In [2]:
# ### To run this notebook in "user mode" use:

# In terminal:
# ```voila prototype.ipynb```
# or
# ```voila <path-to-this-notebook>```

# ### This should open your browser and run the notebook without showing the code cells.

In [3]:
#maybe use streamlit later on, for a cleaner interface

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.utils import GradientDescent

In [5]:
X_MIN = -2
X_MAX = 2
# a_0 = -1.5
eta = 0.1
# #max_iter = 15
# GD = GradientDescent(X_MIN, X_MAX)
# GD.set_a_0(a_0)
# GD.set_eta(eta)
# df_gd = GD.gradient_descent()

In [6]:
#GD.plot_iterations_and_loss()

In [7]:
match_elements = {'aₙ': 'a_n', 
                  '∇f(': 'self.grad_f(', 
                  'f(' : 'self.f(', 
                  ')' : ')', 
                  'η': 'self.eta', 
                  '+': '+', 
                  '-': '-', 
                  '÷' : '/', 
                  '⨉': '*'}

def extract_guessed_formula(string_formula):
    assert len(string_formula.split('(')) == len(string_formula.split(')')), "Be careful! Your formula has unmatched parenthesis."
    
    elements = string_formula.split()
    matched = [match_elements[element] for element in elements]
    
    return ' '.join(matched)
    

## Finding the formula of gradient descent!

### Oh no! You fell asleep during your Machine Learning class and were not able to write down the formula of this method called "Gradient Descent"… You remember a few things about it though:

- This method is used a lot in Machine Learning to find the minimum value of a function. 
- Gradient Descent is a recursive algorithm. It means that you need to provide an initial term (a₀ = …) and the iterative term (aₙ₊₁ = …).
- This method uses the gradient (or derivative) of the function you are trying to find the minimum of.
- It uses a fixed variable called the learning rate η to define how far

### You have an exam in a few days!! You need to find this formula back at ALL COST!!!

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random as random

# Words for the buttons
words = ["aₙ", "∇f(", "f(", ")", "η", "+", "-", "÷", "⨉"]

# Dictionary to track how many times each button was clicked
click_counts = {w: 0 for w in words}
max_clicks = 2

# Numeric box for initial term
init_box = widgets.BoundedFloatText(
    max = X_MAX,
    min = X_MIN,
    value=1.5,
    step = 0.1,
    description="a₀ = ", 
    layout=widgets.Layout(width="25%")
)

# Random button
random_btn = widgets.Button(description="Random", width="25%")

def set_random_init(b):
    init_box.value = round(random.uniform(X_MIN, X_MAX), 1)

random_btn.on_click(set_random_init)

# Put init_box and random_btn side by side
init_ui = widgets.HBox([init_box, random_btn])

# Text area to display the resulting string
output_box = widgets.Textarea(
    value="",
    placeholder="Try and find the formula for gradient descent...",
    description="aₙ₊₁ = ",
    layout=widgets.Layout(width="50%", height="60px")
)

# Function to update the text when a button is clicked
def make_button_callback(word):
    def on_click(b):
        if click_counts[word] < max_clicks:
            click_counts[word] += 1
            output_box.value += (word + " ")
        else:
            print(f"'{word}' already used {max_clicks} times.")
    return on_click

# Create word buttons and attach callbacks
buttons = []
for word in words:
    btn = widgets.Button(description=word)
    btn.on_click(make_button_callback(word))
    buttons.append(btn)

# Restart button
restart_btn = widgets.Button(description="⌫ Erase all", button_style="danger")

def restart_callback(b):
    global click_counts
    click_counts = {w: 0 for w in words}  # reset counts
    output_box.value = ""  # clear text

restart_btn.on_click(restart_callback)

# Simulation button
simulation_btn = widgets.Button(description="Let's try this one!", button_style="success")


# def run_simulation(b):
#     b.guess_formula = output_box.value
#     print("Formula:", b.guess_formula)
#     GD = GradientDescent(X_MIN, X_MAX)
#     GD.set_a_0(init_box.value)
#     GD.set_eta(eta)
#     df_gd = GD.gradient_descent()
#     GD.plot_iterations_and_loss()
    

# simulation_btn.on_click(run_simulation)

# # Layout: output box on top, then buttons, then restart button
# ui = widgets.VBox([init_ui, output_box, widgets.HBox(buttons), restart_btn, simulation_btn])
# display(ui)


output_display = widgets.Output()

def run_simulation(b):
    with output_display:
        clear_output()  # clear previous results
        guess_formula = extract_guessed_formula(output_box.value)
        #print("Formula:", guess_formula)

        GD = GradientDescent(X_MIN, X_MAX)
        GD.set_a_0(init_box.value)
        GD.set_eta(eta)
        df_gd = GD.gradient_descent(guess_formula)
        GD.plot_iterations_and_loss()

simulation_btn.on_click(run_simulation)

# Add output_display to your layout
ui = widgets.VBox([
    init_ui,
    output_box,
    widgets.HBox(buttons),
    restart_btn,
    simulation_btn,
    output_display
])

display(ui)



In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random as random



# Numeric box for initial term
init_box = widgets.BoundedFloatText(
    max = X_MAX,
    min = X_MIN,
    value=1.5,
    step = 0.1,
    description="a₀ = ", 
    layout=widgets.Layout(width="25%")
)

# Random button
random_btn = widgets.Button(description="Random", width="25%")

def set_random_init(b):
    init_box.value = round(random.uniform(X_MIN, X_MAX), 1)

random_btn.on_click(set_random_init)

# Put init_box and random_btn side by side
init_ui = widgets.HBox([init_box, random_btn])



# Slider button for eta
eta_slider = widgets.FloatSlider(
    value=0.8,
    min=0.01,
    max=1.0,
    step=0.01,
    description='η:',
    continuous_update=False,
    layout=widgets.Layout(width="50%")
)





# Simulation button
simulation_btn = widgets.Button(description="Let's try this one!", button_style="success")



output_display = widgets.Output()

def run_simulation(b):
    with output_display:
        clear_output()  # clear previous results
        GD = GradientDescent(X_MIN, X_MAX)
        
        # User inputs
        GD.set_a_0(init_box.value)
        GD.set_eta(eta_slider.value)
        
        df_gd = GD.gradient_descent()
        GD.plot_iterations_and_loss()

simulation_btn.on_click(run_simulation)

# Add output_display to your layout
ui = widgets.VBox([
    init_ui,
    eta_slider,
    simulation_btn,
    output_display
])

display(ui)

